## Using Cloud TPU and a Local runtime in Colab

Make preparations according to Appendix C, Chapter 8, or the following links:

* Creating a Cloud TPU https://cloud.google.com/tpu/docs/managing-tpus-tpu-vm#tpu-vms

* Preparing Jupyter and connect to a Local runtime https://research.google.com/colaboratory/local-runtimes.html


In [1]:
!pip install 'jax[tpu]' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/libtpu_releases.html


Needed for pretty sharding visualization

In [2]:
!pip install rich

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |████████████████████████████████| 239 kB 5.0 MB/s 
     |████████████████████████████████| 87 kB 9.8 MB/s 


In [3]:
import jax
import jax.numpy as jnp

In [4]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [5]:
jax.__version__

'0.4.13'

## Using Tensor Sharding

In [6]:
from jax.experimental import mesh_utils
from jax.sharding import PositionalSharding, NamedSharding

In [7]:
from jax import random

In [8]:
import numpy as np

### Dot example

In [9]:
def dot(v1, v2):
  return jnp.vdot(v1, v2)

In [10]:
rng_key = random.PRNGKey(42)

In [11]:
vs = random.normal(rng_key, shape=(2_000_000,100))
v1s = vs[:1_000_000,:]
v2s = vs[1_000_000:,:]

v1s.shape, v2s.shape

((1000000, 100), (1000000, 100))

In [12]:
jax.debug.visualize_array_sharding(v1s)

┌───────┐
│       │
│       │
│       │
│       │
│ TPU 0 │
│       │
│       │
│       │
│       │
└───────┘

### Positional sharding

In [13]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((8,1)))

In [14]:
sharding

PositionalSharding([[{TPU 0}]
                    [{TPU 1}]
                    [{TPU 2}]
                    [{TPU 3}]
                    [{TPU 6}]
                    [{TPU 7}]
                    [{TPU 4}]
                    [{TPU 5}]])

In [15]:
v1sp = jax.device_put(v1s, sharding)

In [16]:
type(v1sp)

jaxlib.xla_extension.ArrayImpl

In [17]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 5 │
└───────┘

In [18]:
v2sp = jax.device_put(v2s, sharding)

In [19]:
jax.debug.visualize_array_sharding(v2sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 5 │
└───────┘

Input is sharded across all the devices.

In [20]:
d = jax.vmap(dot)(v1sp, v2sp)

In [21]:
d.shape

(1000000,)

In [22]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ TPU 0 │ TPU 1 │ TPU 2 │ TPU 3 │ TPU 6 │ TPU 7 │ TPU 4 │ TPU 5 │
└───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

In [23]:
%timeit jax.vmap(dot)(v1sp, v2sp).block_until_ready()

1.87 ms ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [24]:
%timeit jax.vmap(dot)(v1s, v2s).block_until_ready()

3.91 ms ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
jax.make_jaxpr(jax.vmap(dot))(v1s, v2s)

{ lambda ; a:f32[1000000,100] b:f32[1000000,100]. let
    c:f32[1000000] = dot_general[dimension_numbers=(([1], [1]), ([0], [0]))] a b
  in (c,) }

In [26]:
modules = jax.jit(jax.vmap(dot)).lower(v1s, v2s).compile().compiler_ir()
for hlo in modules:
  print(hlo.to_string())

HloModule jit_dot, is_scheduled=true, entry_computation_layout={(f32[1000000,100]{0,1:T(8,128)}, f32[1000000,100]{0,1:T(8,128)})->f32[1000000]{0:T(1024)}}, allow_spmd_sharding_propagation_to_output={true}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[1000000,100], param_1.2: f32[1000000,100]) -> f32[1000000] {
  %param_0.2 = f32[1000000,100]{0,1:T(8,128)} parameter(0)
  %param_1.2 = f32[1000000,100]{0,1:T(8,128)} parameter(1)
  %multiply.1 = f32[1000000,100]{0,1:T(8,128)} multiply(f32[1000000,100]{0,1:T(8,128)} %param_0.2, f32[1000000,100]{0,1:T(8,128)} %param_1.2)
  %constant.1 = f32[]{:T(256)} constant(0)
  ROOT %reduce.1 = f32[1000000]{0:T(1024)} reduce(f32[1000000,100]{0,1:T(8,128)} %multiply.1, f32[]{:T(256)} %constant.1), dimensions={1}, to

In [27]:
jax.make_jaxpr(jax.vmap(dot))(v1sp, v2sp)

{ lambda ; a:f32[1000000,100] b:f32[1000000,100]. let
    c:f32[1000000] = dot_general[dimension_numbers=(([1], [1]), ([0], [0]))] a b
  in (c,) }

In [28]:
modules = jax.jit(jax.vmap(dot)).lower(v1sp, v2sp).compile().compiler_ir()
for hlo in modules:
  print(hlo.to_string())

HloModule jit_dot, is_scheduled=true, entry_computation_layout={(f32[125000,100]{0,1:T(8,128)}, f32[125000,100]{0,1:T(8,128)})->f32[125000]{0:T(1024)}}, allow_spmd_sharding_propagation_to_output={true}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[125000,100], param_1.2: f32[125000,100]) -> f32[125000] {
  %param_0.2 = f32[125000,100]{0,1:T(8,128)} parameter(0)
  %param_1.2 = f32[125000,100]{0,1:T(8,128)} parameter(1)
  %multiply.2 = f32[125000,100]{0,1:T(8,128)} multiply(f32[125000,100]{0,1:T(8,128)} %param_0.2, f32[125000,100]{0,1:T(8,128)} %param_1.2)
  %constant.2 = f32[]{:T(256)} constant(0)
  ROOT %reduce.2 = f32[125000]{0:T(1024)} reduce(f32[125000,100]{0,1:T(8,128)} %multiply.2, f32[]{:T(256)} %constant.2), dimensions={1}, to_apply=%scala

In [29]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((4,2)))

In [30]:
v1sp = jax.device_put(v1s, sharding)
v2sp = jax.device_put(v2s, sharding)

In [31]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [32]:
d = jax.vmap(dot)(v1sp, v2sp)
d.shape

(1000000,)

In [33]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┬───────┬───────┐
│TPU 0,1│TPU 2,3│TPU 6,7│TPU 4,5│
└───────┴───────┴───────┴───────┘

In [34]:
%timeit jax.vmap(dot)(v1sp, v2sp).block_until_ready()

2.05 ms ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%timeit jax.vmap(dot)(v1s, v2s).block_until_ready()

3.98 ms ± 30 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Looking at HLO (note the all-reduce operation)

In [36]:
modules = jax.jit(jax.vmap(dot)).lower(v1sp, v2sp).compile().compiler_ir()
for hlo in modules:
  print(hlo.to_string())

HloModule jit_dot, is_scheduled=true, entry_computation_layout={(f32[250000,50]{0,1:T(8,128)}, f32[250000,50]{0,1:T(8,128)})->f32[250000]{0:T(1024)}}, allow_spmd_sharding_propagation_to_output={true}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[250000,50], param_1.2: f32[250000,50]) -> f32[250000] {
  %param_0.2 = f32[250000,50]{0,1:T(8,128)} parameter(0)
  %param_1.2 = f32[250000,50]{0,1:T(8,128)} parameter(1)
  %multiply.2 = f32[250000,50]{0,1:T(8,128)} multiply(f32[250000,50]{0,1:T(8,128)} %param_0.2, f32[250000,50]{0,1:T(8,128)} %param_1.2)
  %constant.2 = f32[]{:T(256)} constant(0)
  ROOT %reduce.2 = f32[250000]{0:T(1024)} reduce(f32[250000,50]{0,1:T(8,128)} %multiply.2, f32[]{:T(256)} %constant.2), dimensions={1}, to_apply=%scalar_add_comp

### Using replication

In [37]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((4,2)))

v1sp = jax.device_put(v1s, sharding.replicate(axis=1))

jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│TPU 0,1│
├───────┤
│TPU 2,3│
├───────┤
│TPU 6,7│
├───────┤
│TPU 4,5│
└───────┘

In [38]:
A = random.normal(rng_key, shape=(10000,2000))
B = random.normal(rng_key, shape=(2000,5000))

In [39]:
Ad = jax.device_put(A, sharding.reshape(4, 2).replicate(1))
Bd = jax.device_put(B, sharding.reshape(4, 2).replicate(0))

In [40]:
jax.debug.visualize_array_sharding(Ad)
jax.debug.visualize_array_sharding(Bd)

┌───────┐
│TPU 0,1│
├───────┤
│TPU 2,3│
├───────┤
│TPU 6,7│
├───────┤
│TPU 4,5│
└───────┘

┌─────────────────────────────┬─────────────────────────────┐
│                             │                             │
│                             │                             │
│                             │                             │
│                             │                             │
│         TPU 0,2,4,6         │         TPU 1,3,5,7         │
│                             │                             │
│                             │                             │
│                             │                             │
│                             │                             │
└─────────────────────────────┴─────────────────────────────┘

In [41]:
Cd = jnp.dot(Ad, Bd)

In [42]:
jax.debug.visualize_array_sharding(Cd)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [43]:
C = A @ B

In [44]:
jax.numpy.array_equal(C,Cd)

Array(False, dtype=bool)

In [45]:
jax.debug.visualize_array_sharding(C)

┌──────────┐
│          │
│          │
│          │
│          │
│  TPU 0   │
│          │
│          │
│          │
│          │
└──────────┘

In [46]:
C.shape, Cd.shape

((10000, 5000), (10000, 5000))

In [47]:
C[12,3], Cd[12,3]

(Array(43.027637, dtype=float32), Array(43.027634, dtype=float32))

In [48]:
Ca = jnp.dot(A, B)

In [49]:
jax.numpy.array_equal(C,Ca)

Array(True, dtype=bool)

In [50]:
jax.debug.visualize_array_sharding(Ca)

┌──────────┐
│          │
│          │
│          │
│          │
│  TPU 0   │
│          │
│          │
│          │
│          │
└──────────┘

In [51]:
d = (Cd - C)

In [52]:
jnp.max(d), jnp.sum(d)

(Array(6.1035156e-05, dtype=float32), Array(-0.01252059, dtype=float32))

In [53]:
%timeit jnp.dot(Ad, Bd).block_until_ready()

2.2 ms ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%timeit jnp.dot(A, B).block_until_ready()

10.5 ms ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%timeit (A@B).block_until_ready()

10.5 ms ± 41.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Using sharding constraints

In [56]:
from jax import jit

@jit
def distributed_mul(a, b):
  ad = jax.lax.with_sharding_constraint(a, sharding.replicate(1))
  bd = jax.lax.with_sharding_constraint(b, sharding.replicate(0))
  return jnp.dot(ad, bd)

In [57]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((4,2)))

In [58]:
jax.debug.visualize_array_sharding(A)
jax.debug.visualize_array_sharding(B)

┌───────┐
│       │
│       │
│       │
│       │
│ TPU 0 │
│       │
│       │
│       │
│       │
└───────┘

┌────────────────────────────────────────────────────────────┐
│                                                            │
│                                                            │
│                                                            │
│                                                            │
│                           TPU 0                            │
│                                                            │
│                                                            │
│                                                            │
│                                                            │
└────────────────────────────────────────────────────────────┘

In [59]:
d = distributed_mul(A, B)

In [60]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [61]:
d.shape

(10000, 5000)

In [62]:
@jit
def nondistributed_mul(a, b):
  return jnp.dot(a, b)

In [63]:
dn = nondistributed_mul(A, B)

In [64]:
jax.debug.visualize_array_sharding(dn)

┌──────────┐
│          │
│          │
│          │
│          │
│  TPU 0   │
│          │
│          │
│          │
│          │
└──────────┘

In [65]:
%timeit distributed_mul(A, B).block_until_ready()

21.4 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
%timeit nondistributed_mul(A, B).block_until_ready()

10.5 ms ± 9.29 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Named sharding

In [67]:
from jax.sharding import Mesh
from jax.sharding import PartitionSpec as P
from jax.sharding import NamedSharding

In [68]:
mesh = Mesh(mesh_utils.create_device_mesh((4,2)), axis_names=('batch', 'features'))
sharding = NamedSharding(mesh, P('batch', 'features'))

In [69]:
v1sp = jax.device_put(v1s, sharding)
v2sp = jax.device_put(v2s, sharding)

jax.debug.visualize_array_sharding(v1sp)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [70]:
d = jax.vmap(dot)(v1sp, v2sp)
d.shape

(1000000,)

### Device placement policy and errors

Different devices:

In [71]:
sharding_a = PositionalSharding(np.array(jax.devices()[:4]).reshape(4,1))
sharding_b = PositionalSharding(np.array(jax.devices()[4:]).reshape(4,1))

In [72]:
sharding_a

PositionalSharding([[{TPU 0}]
                    [{TPU 1}]
                    [{TPU 2}]
                    [{TPU 3}]])

In [73]:
sharding_b

PositionalSharding([[{TPU 4}]
                    [{TPU 5}]
                    [{TPU 6}]
                    [{TPU 7}]])

In [74]:
v1sp = jax.device_put(v1s, sharding_a)
v2sp = jax.device_put(v2s, sharding_b)

In [75]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
└───────┘

In [76]:
jax.debug.visualize_array_sharding(v2sp)

┌───────┐
│ TPU 4 │
├───────┤
│ TPU 5 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
└───────┘

In [78]:
d = jax.vmap(dot)(v1sp, v2sp)

ValueError: ignored

Different order:

In [79]:
sharding_a = PositionalSharding(np.array(jax.devices()).reshape(8,1))
sharding_b = PositionalSharding(np.array(jax.devices()[::-1]).reshape(8,1))

In [80]:
sharding_a, sharding_b

(PositionalSharding([[{TPU 0}]
                     [{TPU 1}]
                     [{TPU 2}]
                     [{TPU 3}]
                     [{TPU 4}]
                     [{TPU 5}]
                     [{TPU 6}]
                     [{TPU 7}]]),
 PositionalSharding([[{TPU 7}]
                     [{TPU 6}]
                     [{TPU 5}]
                     [{TPU 4}]
                     [{TPU 3}]
                     [{TPU 2}]
                     [{TPU 1}]
                     [{TPU 0}]]))

In [81]:
v1sp = jax.device_put(v1s, sharding_a)
v2sp = jax.device_put(v2s, sharding_b)

In [82]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 5 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
└───────┘

In [83]:
jax.debug.visualize_array_sharding(v2sp)

┌───────┐
│ TPU 7 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 5 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 0 │
└───────┘

In [84]:
d = jax.vmap(dot)(v1sp, v2sp)

ValueError: ignored

In [85]:
d = jax.vmap(dot)(v1sp, v2s)

In [86]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ TPU 0 │ TPU 1 │ TPU 2 │ TPU 3 │ TPU 4 │ TPU 5 │ TPU 6 │ TPU 7 │
└───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

## MLP example

### Preparing data

Install these modules if you created a new empty cloud machine

In [87]:
!pip install tensorflow

In [88]:
!pip install tensorflow_datasets

In [89]:
import jax
import tensorflow as tf
import tensorflow_datasets as tfds

data_dir = '/tmp/tfds'

data, info = tfds.load(name="mnist",
                       data_dir=data_dir,
                       as_supervised=True,
                       with_info=True)

data_train = data['train']
data_test  = data['test']

2023-11-02 09:57:52.120608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/grigo/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [90]:
HEIGHT = 28
WIDTH  = 28
CHANNELS = 1
NUM_PIXELS = HEIGHT * WIDTH * CHANNELS
NUM_LABELS = info.features['label'].num_classes
BATCH_SIZE  = 32 # total 60k samples
NUM_DEVICES = jax.device_count()

In [91]:
def preprocess(img, label):
  """Resize and preprocess images."""
  return (tf.cast(img, tf.float32)/255.0), label

train_data = tfds.as_numpy(
    data_train.map(preprocess).batch(NUM_DEVICES*BATCH_SIZE).prefetch(1)
)
test_data  = tfds.as_numpy(
    data_test.map(preprocess).batch(NUM_DEVICES*BATCH_SIZE).prefetch(1)
)

In [92]:
len(train_data)

235

### Preparing MLP

In [93]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.nn import swish, logsumexp, one_hot

In [94]:
LAYER_SIZES = [28*28, 512, 10]
PARAM_SCALE = 0.01

In [95]:
def init_network_params(sizes, key=random.PRNGKey(0), scale=1e-2):
  """Initialize all layers for a fully-connected neural network with given sizes"""

  def random_layer_params(m, n, key, scale=1e-2):
    """A helper function to randomly initialize weights and biases of a dense layer"""
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k, scale) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

init_params = init_network_params(LAYER_SIZES, random.PRNGKey(0), scale=PARAM_SCALE)

In [96]:
def predict(params, image):
  """Function for per-example predictions."""
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = swish(outputs)

  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits

batched_predict = vmap(predict, in_axes=(None, 0))

### Loss and update functions

In [97]:
INIT_LR = 1.0
DECAY_RATE = 0.95
DECAY_STEPS = 5
NUM_EPOCHS  = 20

In [98]:
def loss(params, images, targets):
  """Categorical cross entropy loss function."""
  logits = batched_predict(params, images)
  log_preds = logits - logsumexp(logits) # logsumexp trick https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
  return -jnp.mean(targets*log_preds)

@jit
def update(params, x, y, epoch_number):
  loss_value, grads = value_and_grad(loss)(params, x, y)
  lr = INIT_LR * DECAY_RATE ** (epoch_number / DECAY_STEPS)
  return [(w - lr * dw, b - lr * db)
          for (w, b), (dw, db) in zip(params, grads)], loss_value

### Training loop

In [99]:
@jit
def batch_accuracy(params, images, targets):
  images = jnp.reshape(images, (len(images), NUM_PIXELS))
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == targets)

def accuracy(params, data):
  accs = []
  for images, targets in data:
    accs.append(batch_accuracy(params, images, targets))
  return jnp.mean(jnp.array(accs))

#### 8-way data parallelism

In [100]:
sharding = PositionalSharding(jax.devices()).reshape(8, 1)

In [101]:
import time

params = init_params
for epoch in range(NUM_EPOCHS):
  start_time = time.time()
  losses = []
  for x, y in train_data:
    x = jnp.reshape(x, (len(x), NUM_PIXELS))
    y = one_hot(y, NUM_LABELS)
    x = jax.device_put(x, sharding)
    y = jax.device_put(y, sharding)
    params = jax.device_put(params, sharding.replicate())
    params, loss_value = update(params, x, y, epoch)
    losses.append(jnp.sum(loss_value))
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_data)
  test_acc = accuracy(params, test_data)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set loss {}".format(jnp.mean(jnp.array(losses))))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 1.88 sec
Training set loss 0.6914460062980652
Training set accuracy 0.8648548126220703
Test set accuracy 0.874804675579071
Epoch 1 in 1.02 sec
Training set loss 0.6259850859642029
Training set accuracy 0.8882258534431458
Test set accuracy 0.893847644329071
Epoch 2 in 0.99 sec
Training set loss 0.6164931058883667
Training set accuracy 0.8978224396705627
Test set accuracy 0.901074230670929
Epoch 3 in 1.02 sec
Training set loss 0.6100647449493408
Training set accuracy 0.906527042388916
Test set accuracy 0.908984363079071
Epoch 4 in 1.04 sec
Training set loss 0.6044379472732544
Training set accuracy 0.9142231941223145
Test set accuracy 0.917285144329071
Epoch 5 in 1.01 sec
Training set loss 0.5995732545852661
Training set accuracy 0.9211103320121765
Test set accuracy 0.9248046875
Epoch 6 in 1.01 sec
Training set loss 0.5955986976623535
Training set accuracy 0.9261746406555176
Test set accuracy 0.929394543170929
Epoch 7 in 0.99 sec
Training set loss 0.5924407839775085
Training se

In [102]:
jax.debug.visualize_array_sharding(params[0][0])

┌───────────────────────────────────┐
│                                   │
│                                   │
│                                   │
│                                   │
│        TPU 0,1,2,3,4,5,6,7        │
│                                   │
│                                   │
│                                   │
│                                   │
└───────────────────────────────────┘

#### 4-way data parallelism, 2-way tensor parallelism

In [103]:
sharding = PositionalSharding(jax.devices()).reshape(4, 2)

In [104]:
LAYER_SIZES = [28*28, 10000, 10000, 10]
PARAM_SCALE = 0.01

def init_network_params(sizes, key=random.PRNGKey(0), scale=1e-2):
  """Initialize all layers for a fully-connected neural network with given sizes"""

  def random_layer_params(m, n, key, scale=1e-2):
    """A helper function to randomly initialize weights and biases of a dense layer"""
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k, scale) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

init_params = init_network_params(LAYER_SIZES, random.PRNGKey(0), scale=PARAM_SCALE)

In [105]:
sharded_params = []
for i,(w,b) in enumerate(init_params):
  print(i, w.shape, b.shape)
  if i==0:
    w = jax.device_put(w, sharding.replicate(0))
    b = jax.device_put(b, sharding.replicate(0))
  elif i==1:
    w = jax.device_put(w, sharding.replicate(0))
    b = jax.device_put(b, sharding.replicate(0))
  elif i==2:
    w = jax.device_put(w, sharding.replicate())
    b = jax.device_put(b, sharding.replicate())
  sharded_params.append((w,b))


0 (10000, 784) (10000,)
1 (10000, 10000) (10000,)
2 (10, 10000) (10,)


In [106]:
for (w,b) in init_params:
  jax.debug.visualize_array_sharding(w)
  jax.debug.visualize_array_sharding(b)

┌───────┐
│       │
│       │
│       │
│       │
│ TPU 0 │
│       │
│       │
│       │
│       │
└───────┘

┌───────┐
│ TPU 0 │
└───────┘

┌───────────────────────┐
│                       │
│                       │
│                       │
│                       │
│         TPU 0         │
│                       │
│                       │
│                       │
│                       │
└───────────────────────┘

┌───────┐
│ TPU 0 │
└───────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                    TPU 0                                     │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

┌───────┐
│ TPU 0 │
└───────┘

In [107]:
for (w,b) in init_params:
  jax.debug.visualize_array_sharding(jax.device_put(w, sharding.replicate()))
  jax.debug.visualize_array_sharding(jax.device_put(b, sharding.replicate()))

┌───────────────────┐
│                   │
│                   │
│                   │
│                   │
│TPU 0,1,2,3,4,5,6,7│
│                   │
│                   │
│                   │
│                   │
└───────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

┌───────────────────────┐
│                       │
│                       │
│                       │
│                       │
│  TPU 0,1,2,3,4,5,6,7  │
│                       │
│                       │
│                       │
│                       │
└───────────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                             TPU 0,1,2,3,4,5,6,7                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

In [108]:
for (w,b) in sharded_params:
  jax.debug.visualize_array_sharding(w)
  jax.debug.visualize_array_sharding(b)

┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

┌───────────┬───────────┐
│TPU 0,2,4,6│TPU 1,3,5,7│
└───────────┴───────────┘

┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

┌───────────┬───────────┐
│TPU 0,2,4,6│TPU 1,3,5,7│
└───────────┴───────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                             TPU 0,1,2,3,4,5,6,7                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

In [109]:
import time

params = sharded_params
for epoch in range(NUM_EPOCHS):
  start_time = time.time()
  losses = []
  for x, y in train_data:
    x = jnp.reshape(x, (len(x), NUM_PIXELS))
    y = one_hot(y, NUM_LABELS)
    x = jax.device_put(x, sharding.replicate(1))
    y = jax.device_put(y, sharding.replicate(1))
    #params = jax.device_put(params, sharding.replicate())
    params, loss_value = update(params, x, y, epoch)
    losses.append(jnp.sum(loss_value))
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_data)
  test_acc = accuracy(params, test_data)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set loss {}".format(jnp.mean(jnp.array(losses))))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

  jax.debug.visualize_array_sharding(params[0][0])

Epoch 0 in 6.53 sec
Training set loss 0.6637932062149048
Training set accuracy 0.8757756948471069
Test set accuracy 0.8863281607627869


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 1 in 3.94 sec
Training set loss 0.6224857568740845
Training set accuracy 0.8902149200439453
Test set accuracy 0.8956055045127869


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 2 in 3.96 sec
Training set loss 0.6166571378707886
Training set accuracy 0.8960660099983215
Test set accuracy 0.900097668170929


...

┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 18 in 3.95 sec
Training set loss 0.5874113440513611
Training set accuracy 0.9402315616607666
Test set accuracy 0.9404296875


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 19 in 3.92 sec
Training set loss 0.5864334106445312
Training set accuracy 0.9416278004646301
Test set accuracy 0.941699206829071


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

**!!! Do not forget to shutdown your Cloud TPU, or you'll spend much money on it!!!**